In [ ]:
import os
from typing import Dict, NamedTuple, Union, List, Tuple
from itertools import combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class Config(NamedTuple):
    data_d: str = "../20211015_dsd8fac.csv"
    data_r: str = "../dsd_result.csv"
    dirctory: str = "../"
    savefile: bool = False
    kwarg_savefig: Dict[str, Union[str, int, float]] = {
        "facecolor": "white",
        "dpi": 300,
        "bbox_inches": "tight",
        "pad_inches": 0.05
    }

In [3]:
conf = Config(savefile=True)

In [4]:
os.makedirs(conf.dirctory, exist_ok=True)

In [5]:
# load result data

res_d = pd.read_csv(conf.data_d, index_col=0).iloc[:, :-1]
res_r = pd.read_csv(conf.data_r, index_col=0)

In [6]:
res_d.head()

,本体の長さ,羽の長さ,幅,脚の重ね,脚の止め方,糸,羽の形
脚の長さ,,,,,,,
12,2.5,5,8,1,clip,有り,普通
8,2.0,10,6,3,clip,有り,根本切る
8,2.0,10,6,3,セロテープ,無し,普通
12,1.5,15,8,1,clip,有り,普通
4,2.5,5,4,3,clip,有り,普通


In [7]:
design = res_d.copy().assign(
    idx = pd.Series(
        [i+1 for i in range(len(res_d))],
        index = res_d.index
    )
).reset_index().set_index("idx")

In [8]:
design.head()

,脚の長さ,本体の長さ,羽の長さ,幅,脚の重ね,脚の止め方,糸,羽の形
idx,,,,,,,,
1,12,2.5,5,8,1,clip,有り,普通
2,8,2.0,10,6,3,clip,有り,根本切る
3,8,2.0,10,6,3,セロテープ,無し,普通
4,12,1.5,15,8,1,clip,有り,普通
5,4,2.5,5,4,3,clip,有り,普通


In [9]:
arg_d = {
    col: {
        cond: val for cond, val in zip(
            np.sort(design[col].unique()),
            np.arange(-1, 2, 4 - len(design[col].unique()))
        )
    } for col in design.columns
}

l_col = [
    "pod", "body", "wing", "width",
    "thickness", "fixation", "thread", "shape"
]

arg_col = {
    col: v for col, v in zip(design.columns, l_col)
}

In [10]:
readme = f"""
変換内容：
{arg_d}

columnsの変換：
{arg_col}
"""

In [11]:
if conf.savefile:
    with open(f"{conf.dirctory}/README.md", "w") as f:
        f.write(readme)
        
arg_d

{'脚の長さ': {4: -1, 8: 0, 12: 1},
 '本体の長さ': {1.5: -1, 2.0: 0, 2.5: 1},
 '羽の長さ': {5: -1, 10: 0, 15: 1},
 '幅': {4: -1, 6: 0, 8: 1},
 '脚の重ね': {1: -1, 3: 0, 5: 1},
 '脚の止め方': {'clip': -1, 'セロテープ': 1},
 '糸': {'有り': -1, '無し': 1},
 '羽の形': {'普通': -1, '根本切る': 1}}

In [12]:
def converter(
    df: pd.core.frame.DataFrame,
    arg: Dict[str, Dict[Union[int, float, str], int]],
    new_col: List[str]
) -> pd.core.frame.DataFrame:

    flat = df.values.ravel()
    l_conv = [
        arg[
            df.columns[i % df.shape[1]]
        ][v] for i, v in enumerate(flat)
    ]
    df_conv = pd.DataFrame(
        np.array(l_conv).reshape(df.shape),
        index = df.index,
        columns = new_col
    )
    return df_conv
    

In [13]:
d_mat = converter(design, arg_d, l_col)

In [14]:
d_mat.head()

,pod,body,wing,width,thickness,fixation,thread,shape
idx,,,,,,,,
1,1,1,-1,1,-1,-1,-1,-1
2,0,0,0,0,0,-1,-1,1
3,0,0,0,0,0,1,1,-1
4,1,-1,1,1,-1,-1,-1,-1
5,-1,1,-1,-1,0,-1,-1,-1


In [15]:
df = pd.concat([res_r.copy().sort_index(), d_mat], axis=1)

df.head()

,time,pod,body,wing,width,thickness,fixation,thread,shape
1,8.000,1,1,-1,1,-1,-1,-1,-1
2,5.556,0,0,0,0,0,-1,-1,1
3,8.660,0,0,0,0,0,1,1,-1
4,4.620,1,-1,1,1,-1,-1,-1,-1
5,3.620,-1,1,-1,-1,0,-1,-1,-1


In [16]:
if conf.savefile:
    df.to_csv(f"{conf.dirctory}/dsd_result_formatted.csv", index=True)

---
## 生の数値が乗ったcsvも作成

In [17]:
numerical = design.iloc[:, :5].copy()

numerical.columns = d_mat.columns[:5]

In [18]:
df = pd.concat(
    [
        res_r.copy().sort_index(), 
        numerical, 
        d_mat.iloc[:, 5:]
    ],
    axis=1
)

df.head()

,time,pod,body,wing,width,thickness,fixation,thread,shape
1,8.000,12,2.5,5,8,1,-1,-1,-1
2,5.556,8,2.0,10,6,3,-1,-1,1
3,8.660,8,2.0,10,6,3,1,1,-1
4,4.620,12,1.5,15,8,1,-1,-1,-1
5,3.620,4,2.5,5,4,3,-1,-1,-1


In [19]:
if conf.savefile:
    df.to_csv(f"{conf.dirctory}/dsd_result_raw.csv", index=True)